In [1]:
import sys
sys.path.append("C:/Users/Jia-Wei Chang/hu_file/ResNet")
#自定義資料取得
import ESC50_puliic_getDataset as epg
#自定義模組
import ESC50_puliic_model as epm

#資料處理套件
import numpy as np
#畫圖套件
import matplotlib.pyplot as plt
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

In [2]:
#測試模型的準確度
def test_model(inputs_shape, test_loader):
    #固定模型不改變權重
    model.eval() 
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.view(inputs_shape)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += 1
            correct += (predicted == labels).sum().item()

        print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
        return 100 * correct / total

#訓練模型
def train_model(model, loss_func, optimizer, input_shape, num_epochs, train_loader, test_loader1, test_loader3, test_loader5):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    test1_accuracy_array = []
    test3_accuracy_array = []
    test5_accuracy_array = []
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #包裝參數 重組形狀
            train = Variable(inputs.view(input_shape))
            labels = Variable(labels)
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(train)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        #測試電鋸資料集 1S 3S 5S
        test1_accuracy_array.append(test_model((-1,1,128,87), test_loader1))
        test3_accuracy_array.append(test_model((-1,1,128,259), test_loader3))
        test5_accuracy_array.append(test_model((-1,1,128,431), test_loader5))
    return train_loss_array, train_accuracy_array, test1_accuracy_array, test3_accuracy_array, test5_accuracy_array

In [3]:
train_loss_array = []
train_accuracy_array = []
test1_accuracy_array = []
test3_accuracy_array = []
test5_accuracy_array = []
#----------------------------------
#訓練相關參數
#指定訓練位置
device = torch.device('cuda:0')
#訓練次數
num_epochs = 30
#聲音長度
sound = [1, 3, 5]
#批次大小
BATCH_SIZE = 64
#分類數量
num_class = 10
#學習率
LR = 0.0002
#輸入形狀
input_shape = [(-1,1,128,87), (-1,1,128,259), (-1,1,128,431)]
#訓練秒數 1S-> 0 3S-> 1 5S-> 2
trainMode = 0
#----------------------------------
#設定存檔位置
f = open("train.txt", "w")
#k-fold訓練
for fold in range(1,6):
    print("----------------------------------------------------------------------------------------------------------------")
    print("fold: ", fold)
    print("----------------------------------------------------------------------------------------------------------------")
    #建立模型
    model = epm.ResNet_18(nums_class = num_class)
    #指定訓練位置
    model.to(device)
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    #loss方法
    loss_func = nn.CrossEntropyLoss()

    #資料抓入
    train, test_loader1, test_loader3, test_loader5 = epg.getTrainDataset_val135(fold, sound[trainMode], num_class)

    #分批方法
    train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
    
    #訓練模型
    train_loss, train_accuracy, test1_accuracy, test3_accuracy, test5_accuracy = train_model(model, loss_func, optimizer, input_shape[trainMode], num_epochs, train_loader, test_loader1, test_loader3, test_loader5)
    
    #將gpu資料取回
    train_loss = torch.stack(train_loss,dim=-1).cpu()
    train_accuracy = torch.stack(train_accuracy,dim=-1).cpu()
    
    for i in range(num_epochs):
        # 寫入資料
        f.write("fold: {} Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold, i+1, num_epochs, train_loss[i], train_accuracy[i], test1_accuracy[i], test3_accuracy[i], test5_accuracy[i]))
    f.write("---------------------------------------------------------------------------------------------------------------\n")
    
    train_loss_array.append(train_loss)
    train_accuracy_array.append(train_accuracy)
    test1_accuracy_array.append(test1_accuracy)
    test3_accuracy_array.append(test3_accuracy)
    test5_accuracy_array.append(test5_accuracy)

f.close()

----------------------------------------------------------------------------------------------------------------
fold:  1
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/30 Train_Loss: 1.5332813262939453 Train_acc: 43.750000%
Test accuracy of the model: 49.250000%
Test accuracy of the model: 52.083333%
Test accuracy of the model: 53.750000%
Train Epoch: 2/30 Train_Loss: 1.4721087217330933 Train_acc: 56.750000%
Test accuracy of the model: 52.750000%
Test accuracy of the model: 57.083333%
Test accuracy of the model: 53.750000%
Train Epoch: 3/30 Train_Loss: 1.134894609451294 Train_acc: 64.812500%
Test accuracy of the model: 58.500000%
Test accuracy of the model: 64.166667%
Test accuracy of the model: 65.000000%
Train Epoch: 4/30 Train_Loss: 0.9257009029388428 Train_acc: 70.312500%
Test accuracy of the model: 61.750000%
Test accuracy of the model: 66.666667%
Test accuracy of the model: 66.250000%
Train Epoch: 5

Train Epoch: 12/30 Train_Loss: 0.5779549479484558 Train_acc: 82.750000%
Test accuracy of the model: 75.500000%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 78.750000%
Train Epoch: 13/30 Train_Loss: 0.45817989110946655 Train_acc: 83.375000%
Test accuracy of the model: 82.500000%
Test accuracy of the model: 85.833333%
Test accuracy of the model: 83.750000%
Train Epoch: 14/30 Train_Loss: 0.503305971622467 Train_acc: 83.875000%
Test accuracy of the model: 76.500000%
Test accuracy of the model: 80.416667%
Test accuracy of the model: 78.750000%
Train Epoch: 15/30 Train_Loss: 0.7698202133178711 Train_acc: 86.187500%
Test accuracy of the model: 80.000000%
Test accuracy of the model: 84.583333%
Test accuracy of the model: 81.250000%
Train Epoch: 16/30 Train_Loss: 0.5861853957176208 Train_acc: 86.312500%
Test accuracy of the model: 79.250000%
Test accuracy of the model: 84.583333%
Test accuracy of the model: 83.750000%
Train Epoch: 17/30 Train_Loss: 0.4731469750404358 Train

Test accuracy of the model: 77.250000%
Test accuracy of the model: 85.000000%
Test accuracy of the model: 82.500000%
Train Epoch: 25/30 Train_Loss: 0.38636454939842224 Train_acc: 91.187500%
Test accuracy of the model: 78.250000%
Test accuracy of the model: 84.583333%
Test accuracy of the model: 85.000000%
Train Epoch: 26/30 Train_Loss: 0.32722288370132446 Train_acc: 90.437500%
Test accuracy of the model: 77.000000%
Test accuracy of the model: 82.083333%
Test accuracy of the model: 81.250000%
Train Epoch: 27/30 Train_Loss: 0.2538492679595947 Train_acc: 91.000000%
Test accuracy of the model: 76.250000%
Test accuracy of the model: 85.416667%
Test accuracy of the model: 83.750000%
Train Epoch: 28/30 Train_Loss: 0.21064157783985138 Train_acc: 91.250000%
Test accuracy of the model: 78.000000%
Test accuracy of the model: 84.166667%
Test accuracy of the model: 81.250000%
Train Epoch: 29/30 Train_Loss: 0.3661941587924957 Train_acc: 91.500000%
Test accuracy of the model: 79.250000%
Test accuracy

Test accuracy of the model: 65.750000%
Test accuracy of the model: 70.416667%
Test accuracy of the model: 67.500000%
Train Epoch: 6/30 Train_Loss: 0.6423596739768982 Train_acc: 76.125000%
Test accuracy of the model: 67.750000%
Test accuracy of the model: 67.083333%
Test accuracy of the model: 58.750000%
Train Epoch: 7/30 Train_Loss: 0.661048948764801 Train_acc: 76.437500%
Test accuracy of the model: 63.750000%
Test accuracy of the model: 64.166667%
Test accuracy of the model: 58.750000%
Train Epoch: 8/30 Train_Loss: 0.6756370067596436 Train_acc: 76.750000%
Test accuracy of the model: 71.250000%
Test accuracy of the model: 72.500000%
Test accuracy of the model: 72.500000%
Train Epoch: 9/30 Train_Loss: 0.7777377367019653 Train_acc: 79.500000%
Test accuracy of the model: 68.500000%
Test accuracy of the model: 72.083333%
Test accuracy of the model: 70.000000%
Train Epoch: 10/30 Train_Loss: 0.6813827157020569 Train_acc: 80.750000%
Test accuracy of the model: 71.750000%
Test accuracy of the 

In [4]:
train_loss_last = []
train_accuracy_last = []
test1_accuracy_last = []
test3_accuracy_last = []
test5_accuracy_last = []

#設定存檔位置
f = open("lastData.txt", "w")

#整理出訓練完的數值
for fold in range(5):
    train_loss_last.append(train_loss_array[fold][num_epochs-1])
    train_accuracy_last.append(train_accuracy_array[fold][num_epochs-1])
    test1_accuracy_last.append(test1_accuracy_array[fold][num_epochs-1])
    test3_accuracy_last.append(test3_accuracy_array[fold][num_epochs-1])
    test5_accuracy_last.append(test5_accuracy_array[fold][num_epochs-1])
    # 寫入資料
    f.write("Fold: {} Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(fold+1, train_loss_array[fold][num_epochs-1], train_accuracy_array[fold][num_epochs-1], test1_accuracy_array[fold][num_epochs-1], test3_accuracy_array[fold][num_epochs-1], test5_accuracy_array[fold][num_epochs-1]))
f.write("-----------------------------------------------------------------------------------\n")
# 寫入資料
f.write("Max Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(max(train_loss_last), max(train_accuracy_last), max(test1_accuracy_last), max(test3_accuracy_last), max(test5_accuracy_last)))
f.write("Min Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(min(train_loss_last), min(train_accuracy_last), min(test1_accuracy_last), min(test3_accuracy_last), min(test5_accuracy_last)))
f.write("Avg Train_Loss: {} Train_acc: {:.6f}% Test1_accuracy: {:.6f}% Test3_accuracy: {:.6f}% Test5_accuracy: {:.6f}%\n".format(np.mean(train_loss_last), np.mean(train_accuracy_last), np.mean(test1_accuracy_last), np.mean(test3_accuracy_last), np.mean(test5_accuracy_last)))
f.close()